In [3]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [18]:
df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed3.csv")
#df = pd.read_csv( "/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv", index_col=None, header=0, engine='python' )

In [4]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear has...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve been...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bur...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me s...,non-suicide
9,9,My life is over at years oldHello all I am a y...,suicide


In [11]:
l=[]
for i in range(10000):
    l.append(TextBlob(df.iloc[i,1]).sentiment.polarity)
l1=[]
for i in l:
    if i>=0.2:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:10000,2].tolist()


IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(l1,l2))

NameError: name 'l1' is not defined

In [8]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/aman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

In [10]:

#Generating Embeddings using tokenizer
df=df.iloc[0:100,:]
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)
#Model Building
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 713, 120)          60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 713, 120)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 704)               2323200   
                                                                 
 dense (Dense)               (None, 352)               248160    
                                                                 
 dense_1 (Dense)             (None, 3)                 1059      
                                                                 
Total params: 2,632,419
Trainable params: 2,632,419
Non-trainable params: 0
______________________________________________

2023-02-13 22:39:01.595342: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 17s 4s/step - loss: 1.0640 - accuracy: 0.5125
Epoch 2/20
3/3 [==============================] - 14s 4s/step - loss: 1.2800 - accuracy: 0.4750
Epoch 3/20
3/3 [==============================] - 14s 4s/step - loss: 0.8815 - accuracy: 0.5750
Epoch 4/20
3/3 [==============================] - 14s 4s/step - loss: 0.8657 - accuracy: 0.5750
Epoch 5/20
3/3 [==============================] - 14s 4s/step - loss: 0.7469 - accuracy: 0.5750
Epoch 6/20
3/3 [==============================] - 14s 5s/step - loss: 0.7084 - accuracy: 0.5750
Epoch 7/20
3/3 [==============================] - 14s 4s/step - loss: 0.6473 - accuracy: 0.7500
Epoch 8/20
3/3 [==============================] - 14s 4s/step - loss: 0.6640 - accuracy: 0.6250
Epoch 9/20
3/3 [==============================] - 16s 5s/step - loss: 0.5818 - accuracy: 0.8375
Epoch 10/20
3/3 [==============================] - 15s 4s/step - loss: 0.4718 - accuracy: 0.8125
Epoch 11/20
3/3 [==============================] -

[0.716536819934845, 0.800000011920929]

In [12]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.7165 - accuracy: 0.8000


[0.716536819934845, 0.800000011920929]

In [13]:
print(X_train,len(X_train))
print(X_test,len(X_test))
print(y_train,len(y_train))
print(y_test,len(y_test))

[[  0   0   0 ...  22 120  43]
 [  0   0   0 ...   2  14 404]
 [290 105 106 ... 303  11 357]
 ...
 [  0   0   0 ...  36  24 143]
 [  0   0   0 ... 315  79 118]
 [  0   0   0 ...  69 165 164]] 80
[[  0   0   0 ... 253  89   2]
 [  0   0   0 ...  43 445   1]
 [  0   0   0 ...  98  15  81]
 ...
 [  0   0   0 ...  26 356   4]
 [  0   0   0 ...   3  27  45]
 [  0   0   0 ...  56 118  10]] 20
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.

In [19]:
df_text=df["text"].tolist()
df_class=df["class"].tolist()


In [20]:
wantedRows = df[df["text"].str.split().str.len()>250].index 
print(len(wantedRows))
print(len(df))
df_less_450 =  df["text"].drop(wantedRows, axis = 0)
print(len(df_less_450))

11976
232074
220098


In [21]:
(df_less_450)

0         Ex Wife Threatening SuicideRecently left wife ...
1         weird not get affected compliments coming some...
2         Finally almost never hear bad year ever swear ...
3                         need helpjust help me crying hard
4         lostHello name Adam and struggling years and a...
                                ...                        
232069    you not like rock not going get anything but g...
232070    You you tell many friends and not lonely and e...
232071    pee probably tastes like salty tea someone dra...
232072    usual stuff you find hereI not posting sympath...
232073    still not beaten first boss Hollow Knight foug...
Name: text, Length: 220098, dtype: object

In [24]:
df_less_450=df_less_450.tolist()

AttributeError: 'list' object has no attribute 'tolist'

In [42]:
from transformers import pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenized_sentence = tokenizer.encode(df_less_450, padding=True, truncation=True,max_length=50, add_special_tokens = True)
sentiment_pipeline = pipeline("sentiment-analysis",model=model, tokenizer=tokenizer)
l1=[]
for i in df_less_450[0:100]:
    l1.append(sentiment_pipeline(i))
print(l1)
#print(l1[0]["label"])


[[{'label': 'NEGATIVE', 'score': 0.9813257455825806}], [{'label': 'POSITIVE', 'score': 0.9924561977386475}], [{'label': 'POSITIVE', 'score': 0.9879748821258545}], [{'label': 'NEGATIVE', 'score': 0.9963231086730957}], [{'label': 'NEGATIVE', 'score': 0.9901783466339111}], [{'label': 'NEGATIVE', 'score': 0.9974479675292969}], [{'label': 'NEGATIVE', 'score': 0.9769376516342163}], [{'label': 'NEGATIVE', 'score': 0.7028540372848511}], [{'label': 'NEGATIVE', 'score': 0.9953219294548035}], [{'label': 'NEGATIVE', 'score': 0.9972437620162964}], [{'label': 'NEGATIVE', 'score': 0.9661169648170471}], [{'label': 'NEGATIVE', 'score': 0.9979548454284668}], [{'label': 'NEGATIVE', 'score': 0.8084930777549744}], [{'label': 'NEGATIVE', 'score': 0.992155909538269}], [{'label': 'NEGATIVE', 'score': 0.9987903237342834}], [{'label': 'NEGATIVE', 'score': 0.9949349761009216}], [{'label': 'NEGATIVE', 'score': 0.9968137145042419}], [{'label': 'NEGATIVE', 'score': 0.9996371269226074}], [{'label': 'NEGATIVE', 'scor

In [47]:
print(l1[0][0]["label"])

NEGATIVE


In [48]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently left wife ...,suicide
1,1,weird not get affected compliments coming some...,non-suicide
2,2,Finally almost never hear bad year ever swear ...,non-suicide
3,3,need helpjust help me crying hard,suicide
4,4,lostHello name Adam and struggling years and a...,suicide
5,5,Honetly idkI not know even feel like nothing a...,suicide
6,6,Trigger warning Excuse self inflicted burns kn...,suicide
7,7,ends tonight cannot anymore quit,suicide
8,8,Everyone wants edgy and making me self conscio...,non-suicide
9,9,life years oldHello year old balding male hair...,suicide


In [51]:
l3=[]
for i in l1:
    if i[0]["label"]=="NEGATIVE":
        l3.append("suicide")
    else:
        l3.append("non-suicide")


In [52]:
l2=df_class[0:100]
print(accuracy_score(l2,l3))

0.62
